In [1]:
import category_encoders as ce
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import cross_validate


In [2]:
data = pd.read_csv('data.csv')
data = data.drop(columns=['oaid_hash', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1'])
data

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,0,0,0,1,1
1,2021-09-26 22:54:49.000000,1,1,0,0,1,1,1
2,2021-09-26 23:57:20.000000,2,2,3,0,0,1,1
3,2021-09-27 00:04:30.000000,3,3,0,1,1,1,1
4,2021-09-27 00:06:21.000000,4,4,0,1,0,1,1
...,...,...,...,...,...,...,...,...
15821467,2021-10-02 15:51:35.000000,146,530,0,2,9,1,0
15821468,2021-09-27 22:03:14.000000,12,22,0,1,6,1,0
15821469,2021-10-02 17:41:10.000000,12,1236,0,2,0,1,0
15821470,2021-09-29 00:39:32.000000,967,21,0,0,0,1,0


In [3]:
def analysis(data: pd.DataFrame):
    display(data.describe())
    for column in data.drop(columns=['date_time']).columns:
        print('unique values ' + str(column) + ':', len(data[column].unique()))

analysis(data)

,zone_id,banner_id,campaign_clicks,os_id,country_id,impressions,clicks
count,1.582147e+07,1.582147e+07,1.582147e+07,1.582147e+07,1.582147e+07,15821472.0,1.582147e+07
mean,8.152679e+01,3.816483e+02,6.238540e-01,1.840605e+00,4.346986e+00,1.0,2.668835e-02
std,1.632448e+02,3.959386e+02,9.249152e+00,1.530005e+00,4.317701e+00,0.0,1.611710e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00
25%,1.400000e+01,5.200000e+01,0.000000e+00,1.000000e+00,0.000000e+00,1.0,0.000000e+00
50%,1.900000e+01,2.170000e+02,0.000000e+00,2.000000e+00,4.000000e+00,1.0,0.000000e+00
75%,6.000000e+01,6.110000e+02,0.000000e+00,3.000000e+00,7.000000e+00,1.0,0.000000e+00
max,3.443000e+03,1.632000e+03,8.290000e+02,1.000000e+01,1.600000e+01,1.0,1.000000e+00


unique values zone_id: 3444
unique values banner_id: 1633
unique values campaign_clicks: 822
unique values os_id: 11
unique values country_id: 17
unique values impressions: 1
unique values clicks: 2


Можно игнорировать impressions. Время разобъем по колонкам

In [4]:
def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    data['date_time'] = data['date_time'].apply(lambda x: datetime.strptime(x.split('.')[0],'%Y-%m-%d %H:%M:%S'))
    data = data.drop(columns=['impressions'])
    data['year'] = data['date_time'].dt.year
    data['month'] = data['date_time'].dt.month
    data['day'] = data['date_time'].dt.day
    data['hour'] = data['date_time'].dt.hour
    data['minute'] = data['date_time'].dt.minute
    data['second'] = data['date_time'].dt.second
    return data

data = feature_engineering(data)

Тестовую выборку отсечем по последнему дню

In [5]:
last_day = data['date_time'].max().replace(hour=0, minute=0, second=0, microsecond=0)
print('last_day ' + str(last_day))
train_data = data[data['date_time'] < last_day]
print('train_data size ' + str(len(train_data)))
test_data = data[data['date_time'] >= last_day]
print('test_data size ' + str(len(test_data)))

last_day 2021-10-02 00:00:00
train_data size 13692494
test_data size 2128978


In [6]:
target_column = 'clicks'
drop_columns = ['date_time', 'clicks']

X_train = train_data.drop(columns=drop_columns)
y_train = train_data[target_column]
X_test = test_data.drop(columns=drop_columns)
y_test = test_data[target_column]

Нужно закодировать категориальные признаки, для 'os_id' и 'country_id' можем позволить one-hot-encoding. Для 'banner_id' и 'zone_id' воспользуемся target_encoding

In [7]:
one_hot_encoder = ce.OneHotEncoder(cols=['os_id', 'country_id'])
one_hot_encoder.fit(X_train, y_train)
X_train = one_hot_encoder.transform(X_train, y_train)
X_test = one_hot_encoder.transform(X_test, y_test)

target_encoder = ce.TargetEncoder(cols=['zone_id', 'banner_id'])
target_encoder.fit(X_train, y_train)
X_train = target_encoder.transform(X_train, y_train)
X_test = target_encoder.transform(X_test, y_test)

В качестве бэйзлайна возьмем среднее значение

In [8]:
baseline = train_data[target_column].mean()
baseline = np.ones_like(y_test) * baseline
print('log loss = ', log_loss(y_test, baseline))
print('Roc Auc = ', roc_auc_score(y_test, baseline))

log loss =  0.15486198009919555
Roc Auc =  0.5


In [15]:
def cv(X: pd.DataFrame, y: pd.Series):
    grid = [1e-2, 1e-1, 1, 1e1]
    results = []
    for grid_val in grid:
        cur_model = LogisticRegression(C=grid_val, max_iter=50)
        results.append(np.mean(cross_validate(cur_model, X_train, y_train, scoring=['roc_auc'], cv=3)['test_roc_auc']))
        print(grid_val, results[-1])
    return results[np.argmax(results)]

result = LogisticRegression(C=cv(X_train, y_train), max_iter=50)
result.fit(X_train, y_train)

/Users/vtg1337/Downloads/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/vtg1337/Downloads/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

0.01 0.3589451154401177


/Users/vtg1337/Downloads/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/vtg1337/Downloads/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

0.1 0.34486062201593864


/Users/vtg1337/Downloads/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/vtg1337/Downloads/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

1 0.343489964052152


/Users/vtg1337/Downloads/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/vtg1337/Downloads/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

10.0 0.3433882651106341


/Users/vtg1337/Downloads/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.3589451154401177, max_iter=50)

In [16]:
y_predicted = result.predict_proba(X_test)[:, 1]
print('log loss: ', log_loss(y_test, y_predicted))
print('roc auc: ', roc_auc_score(y_test, y_predicted))

log loss:  0.152446954347855
roc auc:  0.5975206813676645
